In [16]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import ModelCheckpoint

import numpy as np
import pandas as pd

from keras import backend as K
K.set_image_dim_ordering('th')

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
model=Sequential()

model.add(Convolution2D(16,3,3,input_shape=(3, 224, 224), activation='relu'))
model.add(Convolution2D(16,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32,3,3, activation='relu'))
model.add(Convolution2D(32,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(96,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(96,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128,3,3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(96, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop', metrics=['accuracy'])

filepath="weights/weights-improvement-{epoch:02d}-{acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

Mask gen

import itertools

image_datagen=ImageDataGenerator(rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    vertical_flip=True)

mask_datagen=ImageDataGenerator(rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    vertical_flip=True)
seed=1
image_generator=image_datagen.flow_from_directory('data/melanoma_preprocessed/',
                                 class_mode='categorical',
                                 seed=seed,
                                batch_size=32)

mask_generator=image_datagen.flow_from_directory('data/segm_melanoma_preprocessed/',
                                 class_mode='categorical',
                                 seed=seed,
                                batch_size=32)
img_mask_generator=itertools.izip(image_generator, mask_generator)

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=90,
                                   horizontal_flip=True,
                                    vertical_flip=True)

train_generator=train_datagen.flow_from_directory('data/melanoma_preprocessed/',
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_datagen=ImageDataGenerator(rescale=1./255)

test_generator=test_datagen.flow_from_directory('data/Evaluation_melanoma/',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                 class_mode='categorical')

Found 1602 images belonging to 3 classes.
Found 398 images belonging to 3 classes.


In [4]:
model.fit_generator(
        train_generator,
        validation_data=test_generator,
        samples_per_epoch=1024,
        nb_val_samples=320,
        nb_epoch=200,
        class_weight='auto',
        callbacks=callbacks_list)

#{0:27., 1:18., 2:0.}

Epoch 1/200
1024/1024 [==============================] - 307s - loss: 0.9645 - acc: 0.6562 - val_loss: 0.9867 - val_acc: 0.7000
Epoch 2/200
 706/1024 [===================>..........] - ETA: 26s - loss: 0.8734 - acc: 0.6983

KeyboardInterrupt: 

In [9]:
pred_generator=test_datagen.flow_from_directory('data/Evaluation_melanoma/',
                                                     target_size=(224,224),
                                                     batch_size=20,
                                                     class_mode='categorical')

Found 398 images belonging to 3 classes.


In [10]:
model.evaluate_generator(pred_generator, val_samples=20)

[0.87311172485351563, 0.69999998807907104]

In [18]:
print("Test classification rate %0.05f" % model.evaluate_generator(pred_generator, val_samples=20)[1])

imgs,labels=pred_generator.next()
array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions=model.predict(imgs)
#rounded_pred=np.asarray([round(i) for i in predictions])

Test classification rate 0.71053


In [20]:
np.asarray([round(i) for i in predictions])

TypeError: only length-1 arrays can be converted to Python scalars

In [25]:
predictions

array([[ 0.20162122,  0.12093255,  0.67744631],
       [ 0.14535975,  0.06697091,  0.78766936],
       [ 0.21432973,  0.13506249,  0.65060771],
       [ 0.21854511,  0.13935384,  0.64210105],
       [ 0.22669081,  0.14944158,  0.62386763],
       [ 0.22282229,  0.14323844,  0.63393927],
       [ 0.19768454,  0.11656766,  0.6857478 ],
       [ 0.23340608,  0.15743954,  0.60915434],
       [ 0.21085729,  0.13032804,  0.65881473],
       [ 0.22884667,  0.15171821,  0.61943513],
       [ 0.21784183,  0.13835444,  0.64380378],
       [ 0.22900264,  0.15219575,  0.61880165],
       [ 0.21175268,  0.13258712,  0.65566021],
       [ 0.20340033,  0.12272677,  0.67387295],
       [ 0.21637015,  0.13774958,  0.64588028],
       [ 0.21337476,  0.13368231,  0.65294296],
       [ 0.20728749,  0.1263973 ,  0.6663152 ],
       [ 0.22468868,  0.14668158,  0.6286298 ],
       [ 0.20885839,  0.12883499,  0.66230661],
       [ 0.1770803 ,  0.09626762,  0.72665209]], dtype=float32)

In [ ]:
pd.crosstab(labels,rounded_pred)